# 1. Hadarmard matrix can be used as hash targets (hash centers) 
In this tutorial, we introduce how to use Hadamard matrix to generate hash targets (hash centers) for image and video datasets

In [1]:
from scipy.special import comb, perm  #calculate combination
from itertools import combinations
from scipy.linalg import hadamard  # direct import  hadamrd matrix from scipy
import torch
import numpy as np

Construct a hadamard matrix. Each row or coloume can be used as a target for a class of images of videos in hashing.

In [2]:
d = 64 # d is the lenth of hash codes and hash centers, d should be 2^n
ha_d = hadamard(d)   # hadamard matrix 
ha_2d = np.concatenate((ha_d, -ha_d),0)  # can be used as targets for 2*d hash bit

In the next, we give examples for a single-label dataset and a multi-label dataset

## 1.1 Generating Hash targets for single-label dataset: ImageNet
We sample 100 classes from ImageNet in experiment. So we generate 100 hash targets, one target for one spcifical class of data. For single-label datasets: ImageNet, UCF101 and HMDB51, the process of generating hash targets is the same

In [3]:
num_class = 100

if num_class<=d:
    hash_targets = torch.from_numpy(ha_d[0:num_class]).float()
    print('hash centers shape: {}'. format(hash_targets.shape))
elif num_class>d:
    hash_targets = torch.from_numpy(ha_2d[0:num_class]).float()
    print('hash centers shape: {}'. format(hash_targets.shape))

hash centers shape: torch.Size([100, 64])


In [4]:
# Save the hash targets as training targets
file_name = str(d) + '_imagenet' + '_' + str(num_class) + '_class.pkl'
file_dir = 'data/imagenet/hash_centers/' + file_name
f = open(file_dir, "wb")
torch.save(hash_targets, f)

In [5]:
# Test average Hamming distance between hash targets
b = []
num_class= 100
for i in range(0, num_class):
    b.append(i)
com_num = int(comb(num_class, 2))
c = np.zeros(com_num)
for i in range(com_num):
    i_1 = list(combinations(b, 2))[i][0]
    i_2 = list(combinations(b, 2))[i][1]
    TF = sum(hash_targets[i_1]!=hash_targets[i_2])
    c[i]=TF

In [6]:
# distance between any two hash targets
c

array([32., 32., 32., ..., 32., 32., 32.])

## 1.2 Generating Hash targets for multi-label dataset: NUS_WIDE

We use 21 classes from NUS_WIDE in experiment. We first generate 21 hash targets, one target for one spcifical class of data; Then, we caculate the centroid for multi-label data. For multi-label datasets: COCO and NUS_WIDE, the process of generating hash centers is the same.

In [7]:
num_class = 21
if num_class<=d:
    hash_targets = torch.from_numpy(ha_d[0:num_class]).float()
    print('hash centers shape: {}'. format(hash_targets.shape))
elif num_class>d:
    hash_targets = torch.from_numpy(ha_2d[0:num_class]).float()
    print('hash centers shape: {}'. format(hash_targets.shape))

hash centers shape: torch.Size([21, 64])


In [8]:
# Save the hash targets as training targets
file_name = str(d) + '_nus_wide' + '_' + str(num_class) + '_class.pkl'
file_dir = 'data/nus_wide/hash_centers/' + file_name
f = open(file_dir, "wb")
torch.save(hash_targets, f)

#### Multi_label centers generation for NUS_WIDE
One simple example, if one image in NUS_WIDE has a smantical label as: [1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]
So the image contains three types of objects, including the first, second and the last class. Then we caculate the centroid of the three corresponding centers.

In [9]:
label = torch.tensor([1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1])
three_centers = hash_targets[label==1]
three_centers

tensor([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,
          1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,
          1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,
          1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,
          1., -1.,  1., -1.,  1., -1.,  1., -1.],
        [ 1.,  1.,  1.,  1., -1., -1., -1., -1.,  1.,  1.,  1.,  1., -1., -1.,
         -1., -1., -1., -1., -1., -1.,  1.,  1.,  1.,  1., -1., -1., -1., -1.,
          1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1., -1.,  1.,  1.,
          1.,  1., -1., -1., -1

In [10]:
centroid = three_centers.mean(dim=0)
centroid[centroid>0]=1.0
centroid[centroid<0]=-1.0
centroid

tensor([ 1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1.,  1.,  1.,  1., -1.,
         1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1.,
         1.,  1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1.,  1.,
         1., -1.,  1., -1.,  1.,  1.,  1.,  1.])

# 2. Generating hash targets by sampling from Bernouli distribution

We just give an example for generating hash center for ImageNet@64bit. The generation for other datasets is the same. And the centroid caculation is similarity with the above process.

In [12]:
# random generation
import torch
import random
import numpy as np
import csv
from scipy.special import comb, perm  #calculate combination
from itertools import combinations
hash_targets = []
a = []  # for sampling the 0.5*hash_bit 
b = []  # for calculate the combinations of 51 num_class
num_class = 100
hash_bit = 64


for i in range(0, hash_bit):
        a.append(i)

for i in range(0, num_class):
    b.append(i)
    
for j in range(10000):
    hash_targets = torch.zeros([num_class, hash_bit])
    for i in range(num_class):
        ones = torch.ones(hash_bit)
        sa = random.sample(a, round(hash_bit/2))
        ones[sa] = -1
        hash_targets[i]=ones
    com_num = int(comb(num_class, 2))
    c = np.zeros(com_num)
    for i in range(com_num):
        i_1 = list(combinations(b, 2))[i][0]
        i_2 = list(combinations(b, 2))[i][1]
        TF = torch.sum(hash_targets[i_1]!=hash_targets[i_2])
        c[i]=TF
    print(min(c))
    print(max(c))
    print(np.mean(c))


    if min(c)>=20 and np.mean(c)>=32:  # guarantee the hash center are far away from each other in Hamming space, 20 can be set as 18 for fast convergence
        print(min(c))
        print("stop! we find suitable hash centers")
        break

18.0
48.0
32.02383838383838
18.0
46.0
32.00525252525252
16.0
48.0
32.00646464646464
20.0
46.0
31.986666666666668
20.0
46.0
31.933333333333334
18.0
44.0
31.93010101010101
18.0
46.0
32.01979797979798
16.0
46.0
31.90707070707071
18.0
46.0
31.90141414141414
18.0
46.0
32.05777777777778
18.0
46.0
31.97939393939394
18.0
46.0
32.02383838383838
18.0
46.0
32.08888888888889
18.0
46.0
31.988282828282827
20.0
46.0
32.034747474747476
20.0
stop! we find suitable hash centers


In [13]:
# Save the hash targets as training targets
file_name = str(d) + '_imagenet' + '_' + str(num_class) + '_class_random.pkl'
file_dir = 'data/imagenet/hash_centers/' + file_name
f = open(file_dir, "wb")
torch.save(hash_targets, f)